In [ ]:
# originally we used a maximum length of 70 for the emails, which might be too short
# a length of 350 showed the best accuracy compared to various email lengths
#max_length = 2000
#trainX =pad_sequences(encoded_train_texts,maxlen=max_length_cnn,padding="post",truncating="post")
#testX =pad_sequences(encoded_test_texts,maxlen=max_length_cnn,padding="post",truncating="post")
#trainX_ws =pad_sequences(encoded_train_texts_ws,maxlen=max_length_cnn,padding="post",truncating="post")
#testX_ws =pad_sequences(encoded_test_texts_ws,maxlen=max_length_cnn,padding="post",truncating="post")

In [3]:
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding(vocab_size, 100)(inputs1)
conv1 = Conv1D(filters=48, kernel_size=4, activation='relu')(embedding1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [4]:
inputs2 = Input(shape=(max_length,))
embedding2 = Embedding(vocab_size, 100)(inputs2)
conv2 = Conv1D(filters=48, kernel_size=6, activation='relu')(embedding2)
drop2 = Dropout(0.5)(conv2)
pool2 = MaxPooling1D(pool_size=2)(drop2)
flat2 = Flatten()(pool2)

In [5]:
inputs3 = Input(shape=(max_length,))
embedding3 = Embedding(vocab_size, 100)(inputs3)
conv3 = Conv1D(filters=48, kernel_size=8, activation='relu')(embedding3)
drop3 = Dropout(0.5)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)

In [6]:
merged = concatenate([flat1, flat2, flat3])

In [7]:
dense1 = Dense(10, activation='relu')(merged)
#outputs = Dense(1, activation='sigmoid')(dense1)
outputs = Dense(2, activation='softmax')(dense1)
model_cnn = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

In [9]:
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history_cnn_1 = model_cnn.fit([trainX,trainX,trainX], array(y_train1), validation_data=([testX,testX,testX],y_test1), epochs=50, batch_size=16, verbose=2,callbacks=[EarlyStopping(patience=3),mc])

In [ ]:
#epochs = [1,2,3]
#batch_size = [8,16]
#param_grid = dict(batch_size=batch_size,epochs=epochs)
#grid = GridSearchCV(estimator = model_cnn, param_grid = param_grid, n_jobs=-1,cv=3,scoring="accuracy")
#grid_result = grid.fit(trainX, array(y_train1))
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [11]:
model_cnn.save('model.h5')

In [11]:
print(model_cnn.summary())
#plot_model(model_cnn, to_file='cnn_plot.png', show_shapes=True, show_layer_names=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     1000000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [12]:
# we have to define a predict function that includes the transformation of the list of texts to a vector of numerical values
def predict_cnn(text):
    stem_texts = [stemSentence(x) for x in text]
    encoded_texts = [one_hot(t,vocab_size) for t in stem_texts]
    encoded_input = pad_sequences(encoded_texts,maxlen=max_length,padding="post",truncating="post")
    return model_cnn.predict([encoded_input,encoded_input,encoded_input])

In [ ]:
explainer_anchor_cnn = AnchorText(nlp, predict_cnn)
explainer_lime = LimeTextExplainer(class_names=class_names)